In [1]:
import numpy as np
import xarray as xr
import netCDF4 as nc
import matplotlib.pyplot as plt
import os
import glob
import dask.array as da
import datetime as dt

In [2]:
qtile=0.9
years = [1991, 2020]
il=0
def sstfile(lead,years):
    return f'/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW/newHindcastMonthly/sst_HC_l{il}_{years[0]}_{years[1]}.nc'
def anomfile(mod,lead,years,detr):
    detrstr='_detr' if detr=='detr' else ''
    return f"/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW/newHindcastMonthly/sstAnom{detrstr}_HC{mod}_l{lead}_{years[0]}_{years[1]}.nc"
def mhwfile(lead,years,detr):
    detrstr='_detr' if detr=='detr' else ''
    return f"/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW/newHindcastMonthly/mhw{detrstr}_HC_l{lead}_{years[0]}_{years[1]}.nc"
akeys=[('GN', 'base'), ('C5', 'base'), ('GN', 'detr'), ('C5', 'detr')]
modkeys=['GN','C5']

In [3]:
ff={};anom={}
for imod in modkeys:
    for idet in ('base','detr'):
        ff[(imod,idet)]=xr.open_dataset(anomfile(imod,il,years,idet),chunks={'S':-1,'M':-1,'lat':60,'lon':90})
        anom[(imod,idet)]=ff[(imod,idet)].anom.data.rechunk({2:60,3:90})

In [4]:
nt,nm,ny,nx=np.shape(anom[akeys[0]]) # both models have same number of ensemble members
nt,nm,ny,nx

(360, 20, 180, 360)

In [5]:
print(dt.datetime.now())
qtiles=[qtile,]
mm=np.arange(0,nt)%12
sst_an_thr={ikey:da.zeros((12,1,ny,nx)) for ikey in akeys} #initialize threshold arrays
for ikey in akeys:
    for im in range(0,12):
        ind=(mm==(im-1)%12)|(mm==im)|(mm==(im+1)%12) # 3 month groups running mean
        temp=anom[ikey][ind,...].reshape((int(nt/12*3*nm),ny,nx))
        sst_an_thr[ikey][im,...]=da.apply_along_axis(np.quantile,q=qtiles,axis=0,arr=temp,shape=(len(qtiles),),dtype=np.float64).compute()
        print(dt.datetime.now())

2024-08-14 22:48:40.464917
2024-08-14 22:48:50.881475
2024-08-14 22:49:01.028803
2024-08-14 22:49:11.493435
2024-08-14 22:49:21.762189
2024-08-14 22:49:31.407545
2024-08-14 22:49:41.883853
2024-08-14 22:49:52.003772
2024-08-14 22:50:01.419426
2024-08-14 22:50:12.002212
2024-08-14 22:50:22.637946
2024-08-14 22:50:32.511557
2024-08-14 22:50:42.334066
2024-08-14 22:50:51.883513
2024-08-14 22:51:01.319218
2024-08-14 22:51:10.892012
2024-08-14 22:51:20.853189
2024-08-14 22:51:30.678378
2024-08-14 22:51:40.149744
2024-08-14 22:51:50.589901
2024-08-14 22:52:00.282053
2024-08-14 22:52:09.791635
2024-08-14 22:52:19.217074
2024-08-14 22:52:28.856637
2024-08-14 22:52:38.370076
2024-08-14 22:52:48.126472
2024-08-14 22:52:57.790483
2024-08-14 22:53:07.341108
2024-08-14 22:53:17.026627
2024-08-14 22:53:26.689668
2024-08-14 22:53:35.667819
2024-08-14 22:53:44.831912
2024-08-14 22:53:53.862927
2024-08-14 22:54:03.151516
2024-08-14 22:54:12.083827
2024-08-14 22:54:21.120387
2024-08-14 22:54:30.141155
2

In [6]:
sst_an_thr

{('GN',
  'base'): dask.array<setitem, shape=(12, 1, 180, 360), dtype=float64, chunksize=(12, 1, 180, 360), chunktype=numpy.ndarray>,
 ('C5',
  'base'): dask.array<setitem, shape=(12, 1, 180, 360), dtype=float64, chunksize=(12, 1, 180, 360), chunktype=numpy.ndarray>,
 ('GN',
  'detr'): dask.array<setitem, shape=(12, 1, 180, 360), dtype=float64, chunksize=(12, 1, 180, 360), chunktype=numpy.ndarray>,
 ('C5',
  'detr'): dask.array<setitem, shape=(12, 1, 180, 360), dtype=float64, chunksize=(12, 1, 180, 360), chunktype=numpy.ndarray>}

In [7]:
is_mhw={ikey:da.zeros(np.shape(anom[ikey])) for ikey in akeys} #initialize mhw arrays
dsout={}
for ikey in akeys:
    is_mhw[ikey]=da.where(anom[ikey]>da.tile(sst_an_thr[ikey],(int(nt/12),nm,1,1)),1,0)
    print(is_mhw[ikey].shape)

/home/ords/crd/ccrn/reo000/miniconda3/envs/py39dc2/lib/python3.9/site-packages/dask/array/core.py:4832: PerformanceWarning: Increasing number of chunks by factor of 12
  result = blockwise(


(360, 20, 180, 360)
(360, 20, 180, 360)


/home/ords/crd/ccrn/reo000/miniconda3/envs/py39dc2/lib/python3.9/site-packages/dask/array/core.py:4832: PerformanceWarning: Increasing number of chunks by factor of 12
  result = blockwise(
/home/ords/crd/ccrn/reo000/miniconda3/envs/py39dc2/lib/python3.9/site-packages/dask/array/core.py:4832: PerformanceWarning: Increasing number of chunks by factor of 12
  result = blockwise(


(360, 20, 180, 360)
(360, 20, 180, 360)


/home/ords/crd/ccrn/reo000/miniconda3/envs/py39dc2/lib/python3.9/site-packages/dask/array/core.py:4832: PerformanceWarning: Increasing number of chunks by factor of 12
  result = blockwise(


In [8]:
is_mhw[('GN','base')]

dask.array<where, shape=(360, 20, 180, 360), dtype=int64, chunksize=(12, 1, 60, 90), chunktype=numpy.ndarray>

In [10]:
for dval in ('base','detr'):
    dsout=xr.Dataset(data_vars={'start_time':ff[('GN',dval)].start_time,
                            'mhw':(['S','M','Y','X'],da.concatenate([is_mhw[('GN',dval)],is_mhw[('GN',dval)]],axis=1))},
                coords=dict(lon=ff[('GN',dval)].lon,lat=ff[('GN',dval)].lat,M=np.arange(1,41),S=ff[('GN',dval)].S),)
    f_out=mhwfile(il,years,dval)
    dsout.to_netcdf(f_out,'w')